In [5]:
from mpi4py import MPI
import numpy as np
#Initialize MPI 
comm = MPI.COMM_WORLD
#Get rank of the communicator
rank = comm.Get_rank()
#Size of the process 
size=comm.Get_size()
#Variable Initialization
N = int(16) 
sum_xy = [] 
executiontime=0
#Master Process
if rank != 0:
    start = MPI.Wtime()
    a = comm.recv(source = 0)
    b= comm.recv(source = 0)
    comm.send(a+b, dest = 0, tag= 1)
    end = MPI.Wtime()
    comm.send(end-start,dest=0, tag=2)
    print("Time taken by worker ", rank, "is: ", end - start)
 #Worker Process
else:
    start = MPI.Wtime()
    x = np.random.randint(100, size = N)
    y = np.random.randint(100, size = N)
    #Splitting the dataset depending on the size of the workers
    split_x= np.array_split(x,size)
    split_y= np.array_split(y,size)
if rank == 0:
    #Master's work 
        sum_xy.extend(list(split_x[rank]+split_y[rank]))
for worker in range(1,size):
    #Pointopoint communication to worker
        comm.send(split_x[worker], dest = worker) 
        comm.send(split_y[worker], dest = worker)
        sum_xy.extend(list(comm.recv(source = worker, tag=1)))
        executiontime+=comm.recv(source=worker, tag=2)
end= MPI.Wtime()
executiontime += (end - start)
print("x: ", x)
print("y: ", y)
print("Final result:", sum_xy)
print("Total execution time:", executiontime)

x:  [ 1 73  6 37 44 74  3  2 55 43 17 75 20 38 72 24]
y:  [96 69 79 53 72 12 23 87 99 45 49 94 27  9  9  9]
Final result: [97, 142, 85, 90, 116, 86, 26, 89, 154, 88, 66, 169, 47, 47, 81, 33]
Total execution time: 0.000551699980860576


In [6]:
#Importing libraries
from mpi4py import MPI
import numpy as np

#Initialize MPI
comm = MPI.COMM_WORLD
#Get rank of the worker
rank = comm.Get_rank()
#Get size of the process
size = comm.Get_size()

#Initialize variable
N = int(1e4)
average = None
average_x = []
executiontime = 0

#Master Process
if rank != 0:
    start = MPI.Wtime()    
    a = comm.recv(source = 0)
    comm.send(np.sum(a)/len(a), dest = 0, tag = 1)
    end = MPI.Wtime()
    comm.send(end-start,dest=0,tag=2)
    print("Time taken by worker ",rank,"is: ", end - start)

#Worker Process
else:
    start = MPI.Wtime()
    x = np.random.randint(100,size=N)
    
    #Split data based on number of workers
    split_x = np.array_split(x,size)
    
    #Master's work
    if rank == 0:
        average_x.append(np.sum(split_x[rank])/len(split_x[rank]))
        
    #Worker's part
    for worker in range(1,size):
        comm.send(split_x[worker], dest = worker)
        average_x.append(comm.recv(source = worker,tag=1))
        executiontime+=comm.recv(source=worker,tag=2)
    
    #Average of the vector
    average = np.average(average_x)
    end = MPI.Wtime()
    executiontime += (end - start)
    print("Time taken by worker ",rank,"is: ", end - start)
    print("Total executed time: ", np.sum(executiontime))
    print("\nX:",x)
    print("Average:",average)

Time taken by worker  0 is:  0.001565599988680333
Total executed time:  0.001565599988680333

X: [ 9 81  5 ... 68 15 42]
Average: 49.0182


In [7]:
#Import Library
from mpi4py import MPI
import numpy as np

#Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

N = int(1e4)
VecMult = []
executiontime = 0

if rank != 0:
    start = MPI.Wtime()    
    A = comm.recv(source = 0)
    b = comm.recv(source = 0)
    comm.send(np.matmul(A,b), dest = 0, tag = 1)
    end = MPI.Wtime()
    comm.send(end-start,dest=0,tag=2)
    print("Time taken by worker ",rank,"is: ", end - start)
    
else:
    start = MPI.Wtime()
    A = np.random.randint(100,size=(N,N))
    b = np.random.randint(100,size=(N,1))
    split_A= np.array_split(A,size)
    
    if rank == 0:
        out = np.matmul(split_A[rank],b)
        VecMult.extend(out.flatten().tolist())
        
    for worker in range(1,size):
        comm.send(split_A[worker], dest = worker)
        comm.send(b,dest=worker)
        out = comm.recv(source = worker,tag=1)
        VecMult.extend(out.flatten().tolist())
        executiontime+=comm.recv(source=worker,tag=2)
    end = MPI.Wtime()
    executiontime += (end - start)
    print("Time taken by worker ",rank,"is: ", end - start)
    print("Total executed time: ", np.sum(executiontime))
    print("\nA:",A)
    print("b:",b)
    print("Vector Multiplication:",VecMult)
    print(len(VecMult))

Time taken by worker  0 is:  1.1332059999986086
Total executed time:  1.1332059999986086

A: [[ 4 74 36 ... 40 45 97]
 [24 83 66 ... 43 35 63]
 [47 97 48 ... 95 23 61]
 ...
 [ 9 95 62 ...  5 58 74]
 [17  8 54 ... 91 39 89]
 [24 68  9 ... 63 23 30]]
b: [[20]
 [11]
 [46]
 ...
 [88]
 [ 7]
 [38]]
Vector Multiplication: [24421861, 24472062, 24407932, 24356854, 24612223, 24284206, 24632143, 24313329, 24267870, 24532135, 24166033, 24346602, 24508213, 24322443, 24511675, 24150907, 24645734, 24304055, 24467466, 24782558, 24432425, 24526661, 24313945, 24474944, 24600914, 24476574, 24290830, 24529755, 24417198, 24425128, 24760704, 24375986, 24205604, 24603706, 24340875, 24273641, 24213974, 24281432, 24587342, 24426924, 24253627, 24323651, 24374944, 24604164, 24446483, 24276456, 24409666, 24563985, 24305970, 24521970, 24895543, 24333704, 24593504, 24349516, 24216989, 24383194, 24227175, 24663162, 24578316, 24293828, 24400267, 24642554, 24304637, 24445568, 24410245, 24517422, 24254330, 24423835, 24

In [8]:
#Import Library
from mpi4py import MPI
import numpy as np

#Initialize MPI
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

#Initialize Matrices
A = None
B = None
N = int(1e2)
C = np.zeros((N,N))


if rank != 0:
    B = np.empty((N,N))
else:
    A = np.random.rand(N,N)
    B = np.random.rand(N,N)

#Receiving variable
Arecv = np.empty((int(N/size),N))
start = MPI.Wtime()

#Separate data to all workers
comm.Scatter(A,Arecv,root=0)

#Send a copy of the vector to all workers
comm.Bcast(B,root=0)

#Gather the result from all workers
comm.Gather(np.matmul(Arecv,B),C,root=0)

end = MPI.Wtime()
print("Time taken by worker:",rank,"is: ",end-start)

Time taken by worker: 0 is:  0.17175910001969896
